# Group 12 - Strategy Implementation and Backtesting

## 0. Import packages

If there are any missing packages, you can install them by typing in the code shell:
```terminal
!pip install <package_name>
```
or
```terminal
%pip install <package_name>
```

In [1]:
!pip install yfinance
import pandas as pd
import yfinance as yf
import numpy as np

from typing import List
from matplotlib import pyplot as plt
from numpy.testing import assert_almost_equal, assert_equal

                                              0.0/108.7 kB ? eta -:--:--
     ----------------------                  61.4/108.7 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 108.7/108.7 kB 2.1 MB/s eta 0:00:00
                                              0.0/64.9 kB ? eta -:--:--
     ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
                                              0.0/3.0 MB ? eta -:--:--
     ---                                      0.3/3.0 MB 5.4 MB/s eta 0:00:01
     --------                                 0.6/3.0 MB 8.0 MB/s eta 0:00:01
     ------------------                       1.4/3.0 MB 10.1 MB/s eta 0:00:01
     ---------------------------------------  3.0/3.0 MB 17.5 MB/s eta 0:00:01
     ---------------------------------------- 3.0/3.0 MB 16.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
transformers 2.1.1 requires sentencepiece, which is not installed.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


## 1. Data Preparation
### 1.1. Data Importing
First, we load the data from `yfinance`:

## 2. Trading Algorithm

## 3. Backtesting

## 4. Evaluation